In [456]:
#pip install lightgbm

In [457]:
import os
from datetime import datetime
import numpy as np
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
import lightgbm as lgb

In [458]:
previsores_file = open(f'{os.getcwd()}\\Tratamento e Pre-Processamento\\Dados\\heart_previsores.pkl', 'rb')
previsores = pickle.load(previsores_file)
previsores_file.close()


alvo_file = open(f'{os.getcwd()}\\Tratamento e Pre-Processamento\\Dados\\heart_alvo.pkl', 'rb')
alvo = pickle.load(alvo_file)
alvo_file.close()

In [459]:
previsores_treino, previsores_teste, alvo_treino, alvo_teste = train_test_split(previsores, alvo, test_size = 0.3, random_state = 0)


In [460]:
dataset_treino = lgb.Dataset(previsores_treino, alvo_treino)

In [461]:
parametros = {'num_leaves': 150, 'objective': 'binary', 'max_depth': 2, 'learning_rate': 0.05, 'max_bin': 150}

In [462]:
inicio = datetime.now()
lgbm = lgb.train(parametros, dataset_treino, num_boost_round = 150)
fim = datetime.now()
tempo = fim - inicio
tempo

[LightGBM] [Info] Number of positive: 352, number of negative: 289
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000123 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 373
[LightGBM] [Info] Number of data points in the train set: 641, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.549142 -> initscore=0.197204
[LightGBM] [Info] Start training from score 0.197204


datetime.timedelta(microseconds=15573)

In [463]:
previsoes_lgbm = lgbm.predict(previsores_teste)

In [464]:
previsoes_lgbm.shape

(276,)

In [465]:
for i in range(0,276):
    if(previsoes_lgbm[i] >= 0.5):
        previsoes_lgbm[i] = 1
    else:
        previsoes_lgbm[i] = 0        

In [466]:
print('Acurácia: %.2f%%' % (accuracy_score(alvo_teste, previsoes_lgbm) * 100))

Acurácia: 87.32%


In [467]:
confusion_matrix(alvo_teste, previsoes_lgbm)

array([[104,  17],
       [ 18, 137]], dtype=int64)

In [468]:
print(classification_report(alvo_teste, previsoes_lgbm))

              precision    recall  f1-score   support

           0       0.85      0.86      0.86       121
           1       0.89      0.88      0.89       155

    accuracy                           0.87       276
   macro avg       0.87      0.87      0.87       276
weighted avg       0.87      0.87      0.87       276



In [469]:
previsoes_treino = lgbm.predict(previsores_treino)
previsoes_treino.shape

(641,)

In [470]:
for i in range(0,641):
    if(previsoes_treino[i] >= 0.5):
        previsoes_treino[i] = 1
    else:
        previsoes_treino[i] = 0   

In [471]:
accuracy_score(alvo_treino, previsoes_treino)

0.9017160686427457

In [472]:
confusion_matrix(alvo_treino, previsoes_treino)

array([[247,  42],
       [ 21, 331]], dtype=int64)

In [473]:
#Validação Cruzada

In [474]:
kfold = KFold(n_splits = 30, shuffle = True, random_state = 5)

In [475]:
modelo =  lgb.LGBMClassifier(num_leaves= 150, objective = 'binary', max_depth = 2, learning_rate = 0.05, max_bin = 150)
resultado = cross_val_score(modelo, previsores, alvo, cv = kfold)

[LightGBM] [Info] Number of positive: 489, number of negative: 397
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000070 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 394
[LightGBM] [Info] Number of data points in the train set: 886, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.551919 -> initscore=0.208426
[LightGBM] [Info] Start training from score 0.208426
[LightGBM] [Info] Number of positive: 490, number of negative: 396
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000085 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 397
[LightGBM] [Info] Number of data points in the train set: 886, number of used features: 11
[LightGBM] [Info] [binary:BoostFro

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000094 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 397
[LightGBM] [Info] Number of data points in the train set: 887, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.556933 -> initscore=0.228726
[LightGBM] [Info] Start training from score 0.228726
[LightGBM] [Info] Number of positive: 493, number of negative: 394
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000096 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 395
[LightGBM] [Info] Number of data points in the train set: 887, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.555806 -> initscore=0.224158
[LightGBM] [Info] Star

In [476]:
print('Acurácia Média: %.2f%%' % (resultado.mean() * 100))

Acurácia Média: 85.94%
